## Meta genomics DWV logboek

Bezig geweest met dorado en pod5 werkende krijgen.  

Pod5 format werkende gekregen. 

## Opzet
De data voor dit onderzoek is verkregen op 5 november 2019, en er is een eerder kort onderzoek op 5 and 18 december 2019 naar deze data gedaan. Uit het eerdere onderzoek is gebleken dat de genoomsequentie van DWV 11kb lang is. De data gemaakt tijdens het vorige onderzoek wordt in dit logboek vergeleken met onze eigen data.

Alle door ons gebruikte libraries en dependencies staan in het bijgevoegde environments.yaml, Hier kan een conda environment van gemaakt worden met het volgende commando:
> $ conda env create --name [envname] --file=environments.yml
>
Deze conda environment kan worden geactiveerd met het volgende commando:
> $ conda activate [envname]
>
Vervang in deze commando's '[envname]' met een omgevingsnaam naar keuze.


## Basecalling
Basecalling is het vertalen van de elektrische stroomveranderingen die waren gemeten toen de nucleotiden door de Nanopore gingen, naar nucleobasen (een A,T,C of G). Voor deze vertaling zijn er meerdere tools om uit te kiezen. De snelste basecaller is Dorado, dus we begonnen met deze. \
Na alles te installeren en het te runnen kregen we deze error: 

> No supported chemistry for Flowcell:'FLO-MIN106', Kit:'SQK-RNA001' and Sampling Rate:3012 

Onze data is met een FLO-MIN106 Flowcell gebasecalled.
Volgens een github comment van raronan op de dorado pagina: https://github.com/nanoporetech/dorado/issues/214 Is onze pore versie r9.4.1. 

Het lijkt erop dat dorado ons type flowcell niet ondersteunt. Als we opzoeken of dorado specifiek deze flowcell ondersteunt kunnen we niet direct wat over ons type vinden. Maar wel een kort lijstje aan versies die niet meer ondersteund worden, deze lijst https://github.com/nanoporetech/dorado/issues/650 gaat van r9.5 tot r10.4, onze versie (r0.4.1) is ouder dan dit. Het lijkt erop dat wij niet dorado kunnen gebruiken voor onze data. 

We hebben besloten de basecalling met guppy te doen, deze tool werkt met oude(re) flowcells. Guppy is echter wel minder snel dan Dorado.

### Guppy

Wij hebben guppy gebruikt om de fast5 data opnieuw te basecallen. Hiervoor hebben wij deze rule gebruikt:


In [ ]:
rule guppy: 

input: 

'/commons/Themas/Thema07/metagenomics/DWV/Sick_Beezzz/20191218_1217_MN30556_FAL66274_dd5dacb8/fast5_pass/' 

output: 

'/students/2023-2024/Thema07/Sick_beez/guppy_out/' 

shell: 

'/opt/ont/ont-guppy/bin/guppy_basecaller -i {input} -s {output} -c /opt/ont/ont-guppy/data/rna_r9.4.1_70bps_hac.cfg --min_qscore 7 --recursive -x 'cuda:0' --num_callers 4 --gpu_runners_per_device 4 --chunks_per_runner 1024 --chunk_size 1000 --compress_fastq' 

 

Wij hebben een nieuwere model gebruikt zodat guppy accurater de bases uit de fast5 files kan callen. Wij hebben dit op het graphix1 systeem gerund op een NVIDIA GeForce GTX 1080 Ti. Het basecallen van alle data duurde ongeveer 15 minuten. 

In de home stonden de conda channels, hier stond een URL bij, deze hebben wij verwijderd zodat we weer met guppy kunnen basecallen.  

## Qualtity control


Voor de quality control twijfelden we tussen FastQC en Nanoplot. We hadden meer ervaring met fastqc maar Nanoplot is specifiek voor MinION/Nanopore data gemaakt. Na met beide kort een testrun te doen zijn we op Nanoplot uitgekomen. Nanoplot is makkelijker in gebruik voor MinION/Nanopore data.

### Nanoplot op de oude data
De door ons gebruikte Nanopore data is al eerder gebasecalled in december 2016 met guppy versie 3.2.6+afc8e14. Om te beginnen gingen we de kwaliteit van deze oude data kijken. \
We hebben Nanoplot met pip install geïnstalleerd op het bin systeem.

We hebben NanoPlot geprobeerd te runnen op de oude data. Het was niet mogelijk om nanoplot op onze folder met guppy data te runnen, dus hebben we Nanoplot op 1 bestand toegepast.

In [ ]:
rule nanoplot: 
    input: 
        reads="/homes/haroorda/periode_7/sick_beez/zipje/your_archive.tar.gz" 
    output: 
        html="results_2/nanoplot_reads.html", 
    shell: 
        """ 
        mkdir -p temp_directory && \ 
        tar -zxvf {input.reads} -C temp_directory && \ 
        NanoPlot --fastq temp_directory/*.fastq -o {output.html} && \ 
        rm -rf temp_directory 
        """ 

Dit is niet gelukt, we kregen errors die niet op te lossen waren. Dit kwam omdat we het op de hele folder met de fastq files wilden runen.
Volgens de documentatie had dit moeten kunnen op de fastq files. 
Wel is het gelukt om nanoplot te runnen op 1 file. 

In [ ]:
/commons/docent/Thema07/metagenomics/DWV/Sick_Beezzz/20191218_1217_MN30556_FAL66274_dd5dacb8/guppy_out_fast/pass$ NanoPlot --fastq fastq_runid_61214d90674fb476eeea0df13759443ace338524_0_0.fastq -o ~/Desktop/test_nanoplot 

### Nanoplot op de nieuwe data


Opnieuw 1 bestand met nanoplot gerund, dit ging goed. Het is weer niet gelukt om het op een hele map te runnen. We hebben de oude en nieuwe basecall data vergeleken en we zagen dat de gemiddelde quality van 8.5 naar 9.9 ging.  

Op de data die uit guppy kwam hebben wij Nanoplot gerunned. Hiervoor hebben wij deze code gebruikt: 

In [ ]:
input_folder="/students/2023-2024/Thema07/Sick_beez/guppy_out/pass/" 

counter=0 

for file in "$input_folder"*.fastq.gz; do 

counter_str=$(printf "%04d" $counter) 

NanoPlot --fastq "$file" --outdir /students/2023-2024/Thema07/Sick_beez/Nanoplot/ --prefix "$counter_str" --only-report 

((counter++)) 

done 

Echter dit maakt 60 losse nanoplot html-bestanden. Dus we moeten eerst nog de fastq files die uit guppy komen samenvoegen tot 1 groot fastq bestand. Wat we al wel konden zien in de 60 gemaakte nanoplot bestanden was dat er reads waren die tussen de 10 en 11 duizend baseparen zaten. Dit komt overeen met de lengte van het genoom van DWV. Als wij een bestand hebben gemaakt waar alle reads in staan kunnen wij daarna dus alleen alle reads boven de 10k baseparen selecteren om alleen de DWV genomen op te nemen. 

In [ ]:
cat /students/2023-2024/Thema07/Sick_beez/guppy_out/pass/*.fastq.gz | gunzip -c > /students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq 

gzip /students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq 

Hier worden de files geconcataneerd, dit gebeurt met de cat command. Het is niet heel efficient, maar het werkt wel. De output is 1 grote file met alle fastq files erin. Deze staat in de map big_guppy.  

NanoPlot --fastq "/students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq.gz" --outdir /students/2023-2024/Thema07/Sick_beez/big_nanoplot/ --prefix "big_" --only-report 

In deze code wordt nanoplot gerund op de geconteneerde fastq files. De html file word opgeslagen in de map big_nanoplot.  

DWV heeft ongeveer een lengte van 11kb dus om hier op te selecteren hebben we alle reads boven de 10kb uit het grote bestand gefilterd om dit te doen hebben wij chopper gebruikt hiervoor is deze code gebruikt  

In [ ]:
gunzip -c /students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq.gz | chopper -l 10000 | gzip > /students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz 

Wij hebben alleen op lengte geselecteerd omdat we al eerder bij het basecallen hebben getrimmed op een minimum quality van 7 

Hierna hebben wij op de getrimde data nanoplot gerund  

In [ ]:
NanoPlot --fastq "/students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz" --outdir /students/2023-2024/Thema07/Sick_beez/trimmed_nanoplot/ --prefix "trimmed_" --only-report 

Daarna hebben we de fastq file omgezet naar een fasta file om deze te gebruiken bij clustal omega. Dit lukte niet in snakemake en hebben we moeten doen met terminal code. Dit hebben uiteindelijk gedaan met de volgende command line:

In [ ]:
gunzip -c /students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz | awk '{if(NR%4==1) {printf(">%s\n",substr($0,2));} else if(NR%4==2) print;}' > /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/filtered_reads.fasta 

Over het paasweekend (28 maart – 2 april) hebben wij clustal omega aangezet met 1 thread. Na het paasweekend was deze nog niet klaar.

In [ ]:
clustalo -i /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/fileter_reads.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa


Toen hebben ervoor gekozen om deze command line nog een keer te runnen maar dan met 40 threads in plaats van de default threads, wat 1 bleek te zijn.

In [ ]:
clustalo -i /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/filtered_reads.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa --threads 40


Hij deed hier nu maar 3-4 uur over, dit is een hele verbetering. Echter hadden we geen guide tree optie meegegeven, waardoor we het opnieuw moesten runnen.

In [ ]:
 clustalo -i /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/filtered_reads.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.aln --threads 80 --guidetree-out /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_tree.txt 

Hierboven is de code gerunt met de guide tree optie erbij, ook hebben wij gekozen voor 80 threads zodat de tool nog sneller klaar zou zijn.

### Tree visualisatie 

Om van de newick file naar een zichtbare phylogentische boom to gaan is de tool ete3 gebruikt. 

Deze tool zet de newick file format om in een zichtbare file. De output is een phylogentische boom.  

In [ ]:
from ete3 import Tree 

tree_path = "/students/2023-2024/Thema07/Sick_beez/output_clustalo/output_tree.txt" # Replace with the path to your Newick format file  

tree = Tree(tree_path)

tree.show() 

De boom die hier uit kwam zag er niet zo heel mooi uit, dit komt ook omdat de boom best veel entiteiten heeft. De namen van de virussen zijn ook best lastig en niet te begrijpen. 

### Missmatch en mutatie hotspots
Hier onder is code over de missmatch en mutatiehotspots grafiek. Hiervoor is de ALIGNIO van de BIO module gebruikt.

In [ ]:
from Bio import AlignIO 

import matplotlib.pyplot as plt 

alignment = AlignIO.read("/students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa", "fasta") 

matches = [sum(1 for seq in alignment[:, i] if seq == alignment[0, i]) for i in range(alignment.get_alignment_length())] 

mismatches = [alignment.get_alignment_length() - m for m in matches ] 

plt.figure(figsize=(10, 5)) 

plt.plot(range(1, alignment.get_alignment_length() + 1), mismatches, color='red', label='Mismatches') 

plt.plot(range(1, alignment.get_alignment_length() + 1), matches, color='blue', label='Matches') 

plt.xlabel('Position in Alignment') 

plt.ylabel('Number of Matches/Mismatches') 

plt.title('Mutation Hotspots') 

plt.legend() 

plt.show() 

De grafiek die hieruit komt laat het aantal missmatches en matches per positie zien. Hieruit kan je opmaken waar de mutaties zitten.